In [19]:

import numpy as np
#from keras_tuner.tuners import RandomSearch, BayesianOptimization, Hyperband
import tensorflow as tf
from kerastuner import HyperParameter, HyperParameters
from kerastuner.tuners import RandomSearch, BayesianOptimization, Hyperband


In [20]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
# Normalize the pixel values to the range of [0, 1].
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Add the channel dimension to the images.
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [21]:
def build_model(hp):
    hp_conv_layers = hp.Int("conv_layers", 1, 3, default=3)
    #hp_dropout= hp.Boolean("dropout")
    hp_dropout_rate=hp.Choice("dropout_rate", values=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5]) 
    hp_activation=hp.Choice('activation', values=['relu', 'tanh'])
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'SGD', 'rmsprop'])
    optimizer = tf.keras.optimizers.get(hp_optimizer)
    optimizer.learning_rate = hp.Choice("learning_rate", [0.1, 0.01, 0.001], default=0.01)
    
    inputs = tf.keras.Input(shape=(28, 28, 1))
    x=inputs
   
    for i in range(hp_conv_layers):
        
        x = tf.keras.layers.Conv2D(
            filters=hp.Int("filters_" + str(i), 4, 32, step=4, default=8),
            kernel_size=hp.Int("kernel_size_" + str(i), 3, 5),
            activation=hp_activation,
            padding="same",
        )(x)

    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Flatten()(x)    
    x = tf.keras.layers.Dropout(hp_dropout_rate)(x)
    
    # The last layer contains 10 units,
    # which is the same as the number of classes.
    outputs = tf.keras.layers.Dense(units=10, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    
    model.compile(
        loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer,
    )
    return model


In [22]:
# Initialize the `HyperParameters` and set the values.
hp = HyperParameters()

In [23]:
model = build_model(hp)
# Test if the model runs with our data.
#model(x_train[:100])
# Print a summary of the model.
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 8)         80        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 8)         584       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 8)         584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1568)             

In [24]:
tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=15,
    executions_per_trial=2,
    overwrite=True,
    directory=".",
    project_name="keras_trial",
)

In [25]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Trial 15 Complete [00h 00m 18s]
val_accuracy: 0.8840000033378601

Best val_accuracy So Far: 0.8923500180244446
Total elapsed time: 00h 05m 40s
INFO:tensorflow:Oracle triggered exit


In [26]:
best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model = tuner.get_best_models()[0]

{'conv_layers': 3, 'dropout_rate': 0.3, 'activation': 'tanh', 'optimizer': 'rmsprop', 'learning_rate': 0.001, 'filters_0': 28, 'kernel_size_0': 4, 'filters_1': 4, 'kernel_size_1': 5, 'filters_2': 16, 'kernel_size_2': 4}


In [27]:
best_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 28)        476       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 4)         2804      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 16)        1040      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dropout (Dropout)            (None, 3136)             

In [28]:
tuner.results_summary()

Results summary
Results in .\keras_trial
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_layers: 3
dropout_rate: 0.3
activation: tanh
optimizer: rmsprop
learning_rate: 0.001
filters_0: 28
kernel_size_0: 4
filters_1: 4
kernel_size_1: 5
filters_2: 16
kernel_size_2: 4
Score: 0.8923500180244446
Trial summary
Hyperparameters:
conv_layers: 2
dropout_rate: 0.4
activation: tanh
optimizer: adam
learning_rate: 0.001
filters_0: 20
kernel_size_0: 3
filters_1: 16
kernel_size_1: 4
filters_2: 4
kernel_size_2: 5
Score: 0.8853999972343445
Trial summary
Hyperparameters:
conv_layers: 1
dropout_rate: 0.2
activation: relu
optimizer: rmsprop
learning_rate: 0.001
filters_0: 16
kernel_size_0: 5
filters_1: 4
kernel_size_1: 4
filters_2: 20
kernel_size_2: 3
Score: 0.8840000033378601
Trial summary
Hyperparameters:
conv_layers: 2
dropout_rate: 0.1
activation: relu
optimizer: adam
learning_rate: 0.01
filters_0: 4
kernel_size_0: 3
filters_1: 20
kernel_size_1

In [29]:
hyperband_tuner = Hyperband(
    hypermodel=build_model,
    objective="val_accuracy",
    max_epochs=2,
    factor=3,
    hyperband_iterations=15,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    directory=".r",
    project_name="keras_trial",
    overwrite=True,
)

In [30]:
hyperband_tuner.search(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Trial 30 Complete [00h 00m 15s]
val_accuracy: 0.10000000149011612

Best val_accuracy So Far: 0.8999000191688538
Total elapsed time: 00h 05m 40s
INFO:tensorflow:Oracle triggered exit


In [31]:
hyperband_tuner.results_summary()

Results summary
Results in .r\keras_trial
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_layers: 2
dropout_rate: 0.0
activation: relu
optimizer: adam
learning_rate: 0.001
filters_0: 24
kernel_size_0: 4
filters_1: 32
kernel_size_1: 5
filters_2: 28
kernel_size_2: 4
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8999000191688538
Trial summary
Hyperparameters:
conv_layers: 2
dropout_rate: 0.1
activation: tanh
optimizer: rmsprop
learning_rate: 0.001
filters_0: 28
kernel_size_0: 4
filters_1: 12
kernel_size_1: 4
filters_2: 28
kernel_size_2: 4
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8903999924659729
Trial summary
Hyperparameters:
conv_layers: 2
dropout_rate: 0.0
activation: relu
optimizer: SGD
learning_rate: 0.1
filters_0: 12
kernel_size_0: 4
filters_1: 12
kernel_size_1: 4
filters_2: 12
kernel_size_2: 4
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 0
tune

In [32]:
best_hps = hyperband_tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model_hp = hyperband_tuner.get_best_models()[0]
best_model_hp.summary()

{'conv_layers': 2, 'dropout_rate': 0.0, 'activation': 'relu', 'optimizer': 'adam', 'learning_rate': 0.001, 'filters_0': 24, 'kernel_size_0': 4, 'filters_1': 32, 'kernel_size_1': 5, 'filters_2': 28, 'kernel_size_2': 4, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 24)        408       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        19232     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)

In [33]:
bo_tuner=BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=15,
    num_initial_points=2,
    alpha=0.0001,
    beta=2.6,
    seed=0,
    hyperparameters=None,
    tune_new_entries=True,
    allow_new_entries=True,
    overwrite=True,
    directory=".",
    project_name="keras_trial",
   
)

In [34]:
bo_tuner.search(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Trial 15 Complete [00h 00m 09s]
val_accuracy: 0.8804000020027161

Best val_accuracy So Far: 0.8898000121116638
Total elapsed time: 00h 03m 28s
INFO:tensorflow:Oracle triggered exit


In [37]:
best_hps = bo_tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model_hp = bo_tuner.get_best_models()[0]

{'conv_layers': 1, 'dropout_rate': 0.0, 'activation': 'relu', 'optimizer': 'adam', 'learning_rate': 0.001, 'filters_0': 32, 'kernel_size_0': 3, 'filters_1': 32, 'kernel_size_1': 5, 'filters_2': 32, 'kernel_size_2': 5}


In [38]:
tuner.results_summary()

Results summary
Results in .\keras_trial
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_layers: 3
dropout_rate: 0.3
activation: tanh
optimizer: rmsprop
learning_rate: 0.001
filters_0: 28
kernel_size_0: 4
filters_1: 4
kernel_size_1: 5
filters_2: 16
kernel_size_2: 4
Score: 0.8923500180244446
Trial summary
Hyperparameters:
conv_layers: 2
dropout_rate: 0.4
activation: tanh
optimizer: adam
learning_rate: 0.001
filters_0: 20
kernel_size_0: 3
filters_1: 16
kernel_size_1: 4
filters_2: 4
kernel_size_2: 5
Score: 0.8853999972343445
Trial summary
Hyperparameters:
conv_layers: 1
dropout_rate: 0.2
activation: relu
optimizer: rmsprop
learning_rate: 0.001
filters_0: 16
kernel_size_0: 5
filters_1: 4
kernel_size_1: 4
filters_2: 20
kernel_size_2: 3
Score: 0.8840000033378601
Trial summary
Hyperparameters:
conv_layers: 2
dropout_rate: 0.1
activation: relu
optimizer: adam
learning_rate: 0.01
filters_0: 4
kernel_size_0: 3
filters_1: 20
kernel_size_1

In [39]:
best_hps = bo_tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model = hyperband_tuner.get_best_models()


{'conv_layers': 1, 'dropout_rate': 0.0, 'activation': 'relu', 'optimizer': 'adam', 'learning_rate': 0.001, 'filters_0': 32, 'kernel_size_0': 3, 'filters_1': 32, 'kernel_size_1': 5, 'filters_2': 32, 'kernel_size_2': 5}


In [43]:
hyperband_tuner.search_space_summary()

Search space summary
Default search space size: 11
conv_layers (Int)
{'default': 3, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
dropout_rate (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5], 'ordered': True}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'SGD', 'rmsprop'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.1, 0.01, 0.001], 'ordered': True}
filters_0 (Int)
{'default': 8, 'conditions': [], 'min_value': 4, 'max_value': 32, 'step': 4, 'sampling': None}
kernel_size_0 (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 5, 'step': 1, 'sampling': None}
filters_1 (Int)
{'default': 8, 'conditions': [], 'min_value': 4, 'max_value': 32, 'step': 4, 'sampling': None}
kernel_size_1 (Int)
{'default': None, 'conditions': [], 'mi

In [48]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
best_hps = hyperband_tuner.get_best_hyperparameters()[0]
model = hyperband_tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=30, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/30
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4089 - accuracy: 0.8563 - val_loss: 0.3044 - val_accuracy: 0.8917
Epoch 2/30
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2711 - accuracy: 0.9029 - val_loss: 0.2777 - val_accuracy: 0.9017
Epoch 3/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2252 - accuracy: 0.9194 - val_loss: 0.2667 - val_accuracy: 0.9051
Epoch 4/30
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1949 - accuracy: 0.9300 - val_loss: 0.2394 - val_accuracy: 0.9168
Epoch 5/30
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1705 - accuracy: 0.9382 - val_loss: 0.2454 - val_accuracy: 0.9158
Epoch 6/30
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1485 - accuracy: 0.9459 - val_loss: 0.2589 - val_accuracy: 0.9098
Epoch 7/30
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1296 - accuracy: 0.9533 - val_loss: 0.2662 - val_accuracy:

In [52]:
print(f"""
The hyperparameter search is complete.
The optimal number of conv layes is {best_hps.get('conv_layers')}
The optimal number of filters  in the first densely-connected layer is {best_hps.get('filters_0')} and 
The optimal number of filters  in the second densely-connected layer is {best_hps.get('filters_1')} 
The optimal kernelsize  in the first densely-connected layer is {best_hps.get('kernel_size_0')} and 
The optimal kernelsize  in the second densely-connected layer is {best_hps.get('kernel_size_1')} 
The best dropout rate is {best_hps.get('dropout_rate')} 
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 
The optimal activation is {best_hps.get('activation')}
The best optimizer is {best_hps.get('optimizer')}       
Number of epochs with best val Accurcay of {max(val_acc_per_epoch)} is {best_epoch}
""")


The hyperparameter search is complete.
The optimal number of conv layes is 2
The optimal number of filters  in the first densely-connected layer is 24 and 
The optimal number of filters  in the second densely-connected layer is 32 
The optimal kernelsize  in the first densely-connected layer is 4 and 
The optimal kernelsize  in the second densely-connected layer is 5 
The best dropout rate is 0.0 
The optimal learning rate for the optimizer is 0.001. 
The optimal activation is relu
The best optimizer is adam       
Number of epochs with best val Accurcay of 0.9170833230018616 is 8

